## YouTube Ranking

In [1]:
import requests, time
import pandas as pd 
from bs4 import BeautifulSoup
from selenium import webdriver

- Step 1, 2

In [2]:
url = 'https://youtube-rank.com/board/bbs/board.php?bo_table=youtube&page=1'
res = requests.get(url)
soup = BeautifulSoup(res.text, 'html.parser')

In [3]:
trs = soup.select('tr.aos-init')
len(trs)

0

In [6]:
driver = webdriver.Chrome()
driver.get(url)

In [7]:
soup = BeautifulSoup(driver.page_source, 'html.parser')
trs = soup.select('tr.aos-init')
len(trs)

100

- Step 3

In [8]:
tr = trs[0]
rank = tr.select_one('.rank').get_text().strip()
rank

'1'

In [9]:
category = tr.select_one('.category').get_text().strip()[1:-1]      # '[음악/댄스/가수]'
category

'음악/댄스/가수'

In [10]:
channel = tr.select_one('.subject > h1 > a').get_text().strip()
channel

'BLACKPINK'

In [11]:
subscriber = tr.select_one('.subscriber_cnt').get_text().strip()
subscriber

'9140만'

In [12]:
view = tr.select_one('.view_cnt').get_text().strip()
video = tr.select_one('.video_cnt').get_text().strip()
view, video

('332억2751만', '571개')

In [13]:
# 숫자로 바꿔주는 함수
def convert(s):
    s = s.replace('억','').replace('개','').replace(',','').replace('만','0000')
    return int(s)

In [14]:
convert('332억2751만')

33227510000

- Step 4

In [15]:
data = []
for tr in trs:
    rank = int(tr.select_one('.rank').get_text().strip())
    category = tr.select_one('.category').get_text().strip()[1:-1]
    channel = tr.select_one('.subject > h1 > a').get_text().strip()
    subscriber = convert(tr.select_one('.subscriber_cnt').get_text().strip())
    view = convert(tr.select_one('.view_cnt').get_text().strip())
    video = convert(tr.select_one('.video_cnt').get_text().strip())
    data.append({'순위':rank, '채널명':channel, '카테고리':category,
                 '구독자수':subscriber, '조회수':view, '비디오수':video})

- Step 5. 에러 수정

In [16]:
try:
    for index, tr in enumerate(trs):
        rank = int(tr.select_one('.rank').get_text().strip())
        category = tr.select_one('.category').get_text().strip()[1:-1]
        channel = tr.select_one('.subject > h1 > a').get_text().strip()
        subscriber = convert(tr.select_one('.subscriber_cnt').get_text().strip())
        view = convert(tr.select_one('.view_cnt').get_text().strip())
        video = convert(tr.select_one('.video_cnt').get_text().strip())
except:
    print(index + 1)        # 순위가 출력되게

In [17]:
data = []
for tr in trs:
    rank = int(tr.select_one('.rank').get_text().strip())
    category = tr.select_one('.category').get_text().strip()[1:-1]
    channel = tr.select_one('.subject > h1 > a').get_text().strip()
    subscriber = convert(tr.select_one('.subscriber_cnt').get_text().strip())
    view = convert(tr.select_one('.view_cnt').get_text().strip())
    video = convert(tr.select_one('.video_cnt').get_text().strip())
    data.append({'순위':rank, '채널명':channel, '카테고리':category,
                 '구독자수':subscriber, '조회수':view, '비디오수':video})

- Step 6

In [18]:
data = []
for page in range(1,11):
    url = f'https://youtube-rank.com/board/bbs/board.php?bo_table=youtube&page={page}'
    driver.get(url)
    time.sleep(2)
    soup = BeautifulSoup(driver.page_source, 'html.parser')
    trs = soup.select('tr.aos-init')

    for tr in trs:
        rank = int(tr.select_one('.rank').get_text().strip())
        category = tr.select_one('.category').get_text().strip()[1:-1]
        channel = tr.select_one('.subject > h1 > a').get_text().strip()
        subscriber = convert(tr.select_one('.subscriber_cnt').get_text().strip())
        view = convert(tr.select_one('.view_cnt').get_text().strip())
        video = convert(tr.select_one('.video_cnt').get_text().strip())
        data.append({'순위':rank, '채널명':channel, '카테고리':category,
                    '구독자수':subscriber, '조회수':view, '비디오수':video})
        
driver.close()

- Step 7

In [19]:
df = pd.DataFrame(data)
df.head()

,순위,채널명,카테고리,구독자수,조회수,비디오수
0,1,BLACKPINK,음악/댄스/가수,91400000,33227510000,571
1,2,BANGTANTV,음악/댄스/가수,76400000,21360690000,2336
2,3,HYBE LABELS,음악/댄스/가수,72700000,30124270000,1435
3,4,SMTOWN,음악/댄스/가수,32100000,27903520000,4229
4,5,JYP Entertainment,음악/댄스/가수,27900000,20584660000,1843


In [20]:
df.to_csv('data/유튜브랭킹_20230919.csv', index=False)
pd.read_csv('data/유튜브랭킹_20230919.csv').tail()

,순위,채널명,카테고리,구독자수,조회수,비디오수
995,994,솜이네 곰이탱이여우,애완/반려동물,770000,531880000,1806
996,1000,빅페이스 BIGFACE,미분류,770000,184320000,143
997,999,tweety 트위티,TV/방송,770000,152420000,275
998,998,UTZI웃지,영화/만화/애니,770000,6500000,13
999,997,뉴스데일리베스트,뉴스/정치/사회,770000,248950000,2964
